In [9]:
import pandas as pd


df = pd.read_csv('C:\\Users\\Dar Computers\\Desktop\\10Acadamy\\week 3\\code\\acis-week3-insurance-analysis\\data\\processed\\insurance_cleanedd.csv')


In [13]:
import numpy as np
df['ClaimOccurred'] = df['TotalClaims'] > 0
df['ClaimFrequency'] = df['ClaimOccurred'].astype(int)
df['ClaimSeverity'] = df['TotalClaims'] / df['ClaimFrequency'].replace(0, np.nan)
df['Margin'] = df['TotalPremium'] - df['TotalClaims']


In [15]:
import scipy.stats as stats

province_groups = df.groupby('Province')['ClaimFrequency']
stat, p = stats.f_oneway(*[group for name, group in province_groups])
print(f"Claim Frequency - Provinces: p={p}")


Claim Frequency - Provinces: p=2.8046511194894357e-20


In [21]:
zipcode_groups = df.groupby('PostalCode')['ClaimFrequency']
stat, p = stats.f_oneway(*[group for name, group in zipcode_groups if len(group) > 10])
print(f"Claim Frequency - PostalCode: p={p}")


Claim Frequency - PostalCode: p=2.325860275130524e-34


In [23]:
zipcode_margin = df.groupby('PostalCode')['Margin']
stat, p = stats.f_oneway(*[group for name, group in zipcode_margin if len(group) > 10])
print(f"Margin - PostalCode: p={p}")


Margin - PostalCode: p=0.9626287828864589


In [27]:
# Filter only Male and Female to be safe
filtered = df[df['Gender'].isin(['Male', 'Female'])]

# Group them explicitly
male_freq = filtered[filtered['Gender'] == 'Male']['ClaimFrequency']
female_freq = filtered[filtered['Gender'] == 'Female']['ClaimFrequency']

stat, p = stats.ttest_ind(male_freq, female_freq, equal_var=False, nan_policy='omit')

print(f"Claim Frequency - Gender: t={stat:.4f}, p={p:.4f}")


Claim Frequency - Gender: t=0.2055, p=0.8372


## Analysis and Reporting

Below is the interpretation of the statistical test results for each hypothesis:

| Hypothesis | p-value | Decision | Interpretation |
|-----------|---------|----------|----------------|
| **H₀: There are no risk differences across provinces** | `2.80e-20` | **Reject H₀** ✅ | There is a statistically significant difference in claim frequency across provinces. Some provinces pose higher insurance risks and should be considered for regional pricing adjustments. |
| **H₀: There are no risk differences between zip codes** | `2.33e-34` | **Reject H₀** ✅ | Risk levels vary significantly across postal codes. This supports a more granular segmentation strategy based on zip code. |
| **H₀: There is no significant margin (profit) difference between zip codes** | `0.96` | **Fail to Reject H₀** ❌ | There is no statistically significant difference in profit margin across zip codes. Margin does not vary enough geographically to justify changes in pricing or strategy. |
| **H₀: There is no significant risk difference between Women and Men** | `0.8372` | **Fail to Reject H₀** ❌ | Claim frequency is statistically similar between genders. Gender-based pricing is not supported by the data and may not be legally or ethically appropriate. |

### 📌 Business Recommendations

- **Regional Risk-Based Pricing**: Provinces and postal codes should be used as key variables in pricing models due to their significant influence on claim frequency.
- **Avoid Gender-Based Pricing**: Gender does not significantly influence risk in this dataset and should not be used for segmentation.
- **Postal Code Profitability**: Although claim frequency varies by zip code, margin does not — pricing should focus on risk rather than profit variations by location.
